# Mastering Large Language Models: Efficient Techniques for Fine-Tuning - LAB SESSION

### Some Google Colab essentials

> If you already use Colab and/or know how to setup a runtime environment with GPU resources, you may not learn anything new in this part.

**What is Colab / a Jupyter Notebook?**   
Colab works like a regular Jupyter Notebook. If you are not familiar with such environment, here is a brief overview:
- You can write markdown in `Text` cells.
- You can execute Python code in `Code` cells.
- You can run command-line commands in `Code` cells by starting the line with `!`.
- You can display printed or plotted outputs, which will appear and persist below the corresponding cell.

In Colab, the code is executed on a virtual environment hosted by Google. To fine-tune LLMs using this notebook, we need to set up the appropriate environment.

**Enabling GPU**  
Running this tutorial requires GPU for faster computations. In order to use GPU on Google Colab, you need to connect to a runtime engine that includes enough GPU resources. It is not the case for the default engine provided. To switch to a GPU-enabled engine:
1. Click the downward arrow next to the RAM/Disk display (top right).  
2. Select `Change runtime type`.  
3. Under the `Hardware Accelerator` section, choose **T4 GPU** (available for free).

**Check runtime environment and GPU config**  
All information related to the virtual environment on which you are running your code is available _via_ the right panel `Resources`. Click on the RAM/Disk display to check for disk and CPU/GPU usage history. The logo on the left of this display indicates the state of the engine.

## 0. Setup everything

The following cell checks the usual packages are already installed and installs specific ones for evaluation.

In [1]:
!python3 -m pip install transformers datasets evaluate torch scikit-learn rouge_score plotly peft

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


### Import Python libraries and load models

Attention this might take several minutes, please start to run it early. Model loading approx running time 3m.

In [2]:
import os
import torch
import time
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import f1_score

# HuggingFace librairies
from datasets import load_dataset, load_from_disk
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from evaluate import load

/Users/bgendron/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bgendron/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def activate_gpu(force_cpu=False):
    '''
        A function to return the right device depending on GPU availability
    '''
    device = "cpu"
    if not force_cpu:
        if torch.cuda.is_available():
            device = 'cuda'
            print(f'DEVICE = {torch.cuda.get_device_name(0)}')
        elif torch.backends.mps.is_available():
            device = 'mps'
            print('DEVICE = mps')
        else:
            device = 'cpu'
            print('DEVICE = CPU')
    return device

In [136]:
device = activate_gpu(force_cpu=True)

MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    low_cpu_mem_usage = True,
    return_dict=True,
    torch_dtype = torch.float32,
    )

model.to(device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = tokenizer.eos_token_id

print(model)

  4%|▍         | 28/625 [02:53<1:01:31,  6.18s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

## 1. Prompt LLMs for Text Generation

This parts shows the basics of prompting and handling generated content from an LLM using HuggingFace `pipeline`.

### 1.1 Individual _vs_ Batched Generation

In [ ]:
def individual_text_generation(prompt, temperature=0.6):
    # tokenize the prompt content (in a torch tensor)
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs.to(device) # the input tokens should be on the same device than the model

    # call .generate() method to get the next tokens
    output = model.generate(
        inputs["input_ids"],         #
        attention_mask = inputs["attention_mask"],
        max_new_tokens = 100,             # The maximum length of the generated sequence
        temperature = temperature,   # Controls randomness; higher means more diverse outputs
        do_sample = True,
        top_k = 50,                  #
        top_p = 0.9,                 #
        repetition_penalty = 1.2,    # Penalizes repeating phrases (values > 1 discourage repetition)
        pad_token_id=tokenizer.eos_token_id,
    )

    # get the token strings from the token indexes
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

In [ ]:
def batched_text_generation(prompts, temperature=0.6):
    # create a text generation pipeline with the model and tokenizer
    device_index = 0 if device == "cuda" else -1
    pipe = pipeline(
        "text-generation",
        model = model,
        model_kwargs = {"torch_dtype": torch.bfloat16},
        device_map = 'cpu',
        tokenizer = tokenizer,
    )

    outputs = pipe(
        prompts,
        max_new_tokens = 100,                    # limit the length of the generated content (takes less time to compute)
        min_new_tokens = 10,                    # ensure at least this amount of tokens is generated
        # max_length = 100,                       # max length of prompt + generated content
        eos_token_id = tokenizer.eos_token_id,  # the id of the end-of-sentence token
        pad_token_id=tokenizer.eos_token_id,
        do_sample = True,
        temperature = temperature,              # model "creativity"
        top_k = 50,
        top_p = 0.9,
        repetition_penalty = 1.2,
        num_return_sequences = 1,               # number of generated sequences by prompt
    )

    return outputs

The following cells applies the two generation approaches. You can notice that, by default, the prompt is included in the generated content. You can print only the generating content by setting `return_prompt=False` in ...

In [ ]:
one_prompt = 'What is usually the weather like in Nancy, France?'

several_prompts = [
    'What is the best way to learn music?',
    'Teach me something surprising about artificial intelligence.',
]

In [ ]:
individual_text_generation(one_prompt)

In [ ]:
batched_text_generation(several_prompts)

As you can notice in the above examples, **the prompt is always included in the output**. Consequently, if you aim to further analyse the generated content only, you need to slice the output to exclude the prompt. In practice, post-processing is often necessary when performing fine-tuning because it affects generation format and quality, potentially introducing unwanted characters or structuring responses in unintended ways.

### 1.2 Play with temperature

The temperature parameter introduces random perturbations into the output probabilities, directly affecting the model's creativity and determinism during text generation. A lower temperature (usually, <1) makes the model more deterministic. It is ideal for tasks like summarization or question answering. On the contrary, a higher temperature (>1) broadens the distribution, allowing the model to generate less likely tokens in the given context. This behaviour is often described as model "creativity", but it should be used wisely because a too high temperature would lead to nonsensical content. In the below cell, you can compare generations from the same prompt with different temperatures. 

In [ ]:
for temp in [0.5, 0.7, 0.8, 1.0, 1.1, 1.3, 1.6]:
    print(f"Temperature = {temp}")
    print(individual_text_generation(one_prompt, temperature=temp))
    print(40*'--')
    print('')

## 2. A Baseline Benchmark on the Pre-Trained Model

The following function is to be used at the end of the part to display the result of the benchmark.

In [ ]:
def display_radar_chart(metrics):
    # format data
    df = pd.DataFrame(dict(
        r=metrics,
        theta=['Summarization', 'Translation', 'Classification',
              'Causal Language Modeling', 'Emotion Load Detection']))

    # create plot
    fig = px.line_polar(df, r='r', theta='theta', line_close=True)

    # define colors
    fig.update_traces(
        fill='toself',
        fillcolor='rgba(165, 42, 42, 0.6)',
        line=dict(color='rgb(165, 42, 42)', width=3)
    )

    # layout customization + define scale intervals
    fig.update_layout(
        polar = dict(
            bgcolor = 'rgba(211, 211, 211, 0.2)',
            angularaxis = dict(
                linewidth = 1,
                linecolor = "gray",
                showgrid = True,
                gridcolor = 'lightgray',
                tickfont = dict(size=12),
            ),
            # background and scale customization
            radialaxis = dict(
                visible=True,
                showgrid = True,
                gridcolor = 'lightgray',
                tickvals = [0, 0.2, 0.4, 0.6, 0.8, 1],
                ticktext = ['0', '0.2', '0.4', '0.6', '0.8', '1'],
                tickfont = dict(size=12, color='rgb(165, 42, 42)'),
                linewidth = 1,
                ticks = 'outside',
                linecolor = 'rgb(50, 50, 50)',
                range=[0, 1]  # Set the fixed range from 0 to 1
            )
        ),
        font = dict(
            size = 14
        )
    )

    # display
    fig.show()

### 2.1 Benchmark Tasks

In this section, each task is described with the following information:  
- The name of the task.  
- The dataset used for evaluation, including specific details such as splits, subsets, or relevant columns.  
- The prompt format for generating the required content for the task.  
- The evaluation metric used to quantify performance.  

In [10]:
def structured_prompt(task_info, x):
    if task_info == "summarization":
        return f'''
            Summarize the following conversation in one short, concise paragraph. Please provide only the summary and nothing else.

            Conversation:
            {x['dialogue']}
            Summary:
        '''
    
    elif task_info == "translation":
        return f'''
            Translate the following English text to French. Please provide only the translation and nothing else.
            English:
            {x['en']}
            French:
        '''
    
    elif task_info == "classification":
        return f'''
            Classify the following document into one of the following classes: 'Sale', 'Baseball', 'Graphics', or 'Space'. Provide only the class name as the output, with no additional text.
            Document:
            {x['data']}
            Class:
        '''
    
    elif task_info == "emotion_load_detection":
        return f'''
            You will be given an utterance. Is there an emotional load in this sentence? Generate 'yes' or 'no', nothing else.
            Utterance:
            {x['utterance']}
            Answer:
        '''

In [11]:
tasks = {
    "summarization": {
        "dataset": "../data/samsum",
        "columns": ("article", "highlights"),
        "task": "Dialogue Summarization",
        "prompt": lambda x: structured_prompt("summarization", x),
        "metric": load("rouge"),
    },
   "translation": {
        "dataset": "../data/tatoeba_en_fr",
        "task": "Translation (English to French)",
        "prompt": lambda x: structured_prompt("translation", x),
        "metric": load("bleu"),
    },
    "classification": {
        "dataset": "../data/4newsgroup",
        "task": "Document Classification",
        "prompt": lambda x: structured_prompt("classification", x),
        "metric": f1_score,
    },
    "causal_language_modeling": {
        "dataset": "../data/openwebtext",
        "task": "Causal Language Modeling",
        "prompt": lambda x: f"{x['baseline']}",
        "metric": load("perplexity")
    },
    "emotion_load_detection": {
        "dataset": "../data/emotion_load",
        "task": "Emotion Load Detection",
        "prompt": lambda x: structured_prompt("emotion_load_detection", x),
        "metric": f1_score,
    },
}

The following cells run the benchmark. In order to obtain results in a reasonable amount of time, only 100 samples are studied for each dataset.  
_N.B.:_ You can notice that no structured prompt is provided for Causal Language Modeling (CLM) task. This is because CLM is the task used to pre-train such models, hence they are already design to continue a sentence using autoregressive principle.

In [ ]:
def post_process_generated_output(task_name, output):
    if task_name in ["summarization", "translation"]:
        # remove the content after a blank line
        lines = output.split("\n")
        for i, line in enumerate(lines):
            if line.strip() == "":
                return "\n".join(lines[:i])
    
    if task_name in ["classification", "emotion_load_detection"]:
        # keep only the first word (= 1st succession of characters without spaces) because it corresponds to the predicted class
        return output.split()[0] if output else ""

    else:
        return output

In [ ]:
def evaluate_task(task_name, task_info, model, tokenizer, num_samples=100):
    print(f"\n{10*'-'} Benchmarking {task_info['task']} {10*'-'}")
    dataset = load_from_disk(task_info["dataset"])
    test_data = dataset["test"].shuffle(seed=24).select(range(num_samples))

    # Generate prompts
    examples = test_data.map(
        lambda x: {"prompt": task_info["prompt"](x)},
    )

    # get the name of the ground truth column (the true label or the baseline content) to evaluate generation depending on the task
    ground_truth_column_name = 'label' if task_name in ['classification', 'emotion_load_detection'] else 'baseline'

    # perform evaluation
    results = []
    tic = time.time()
    for idx, example in enumerate(examples):
        prompt = example["prompt"]
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
        baseline = example[ground_truth_column_name]
        # run generation
        output = model.generate(
            inputs['input_ids'],
            attention_mask = inputs['attention_mask'],
            max_new_tokens = 52,
            temperature = 0.8,
            top_p = 0.9,
            do_sample = True,
            repetition_penalty = 1.2,
            pad_token_id=tokenizer.eos_token_id,
        )
        # decode the output and remove the prompt
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_text_without_prompt = generated_text[len(prompt):].strip()
        cleaned_generation = post_process_generated_output(task_name, generated_text_without_prompt)

        # store prompt and generated content to compute metrics afterwards
        results.append((baseline, cleaned_generation))

        # optional: display the baseline (what is expected), and the generated content
        # print(f"Baseline: {baseline}\nOutput: {cleaned_generation}\n")

    toc = time.time()
    print(f"Task {task_name} completed in {toc-tic:.2f}s for {num_samples} samples.")

    return results

In [ ]:
def process_clf_tasks_outputs(results, task_name):
    if task_name == "emotion_load_detection":
        # consider the sentence is non loaded if the generation does not contain any markers of loaded
        trues = [res[0] for res in results]
        preds = [1 if any(word in res[1].lower() for word in ['yes', 'load']) else 0 for res in results]        
        return f1_score(trues, preds)

    else:
        # add 'unknown' class in case nothing corresponds to a given class in the generated content
        class_mapping = {'unknown':0, 'misc.forsale':1, 'rec.sport.baseball':2, 'comp.graphics':3, 'sci.space':4}
        generation_mapping = {'sale':'misc.forsale', 'baseball':'rec.sport.baseball', 'graphics':'comp.graphics', 'space':'sci.space'}
        # it is document classification
        trues = [class_mapping[res[0]] for res in results]
        preds = [class_mapping[generation_mapping[res[1].lower()]] if res[1].lower() in generation_mapping else 0 for res in results]
        # compute f1 score in a multiclass setting (weigthed average across classes)
        return f1_score(trues, preds, average='weighted')

In [ ]:
def evaluate_benchmark(model, tokenizer):
    metrics = {
        'summarization':0,
        'translation':0,
        'classification':0,
        'causal_language_modeling':0,
        'emotion_load_detection':0,
    }
    for task_name, task_info in tasks.items():
        task_info['results'] = evaluate_task(task_name, task_info, model, tokenizer, num_samples=5)
        # load the corresponding metric
        metric = task_info["metric"]

        # check the task name to apply the metric correctly
        if task_name == "causal_language_modeling":
            results = task_info['results']
            # metric is perplexity
            perplexities = []
            
            for prompt, generation in results:
                # Compute perplexity for prompt + generation
                text = prompt + generation
                perplexity = metric.compute(predictions=[text], model_id='gpt2')['mean_perplexity']
                perplexities.append(perplexity)
            
            # per-word perplexity for better interpretability
            normalized_perplexities = [abs(p-np.mean(perplexities))/np.std(perplexities) for p in perplexities]
            metrics['causal_language_modeling'] = float(np.mean(normalized_perplexities))

        elif task_name == "summarization":
            results = task_info['results']
            # metric is rouge
            tokenized_references = [" ".join(tokenizer.tokenize(text)) for text, _ in results]
            tokenized_predictions = [" ".join(tokenizer.tokenize(summary)) for _, summary in results]
            metrics['summarization'] = float(metric.compute(predictions=tokenized_predictions, references=tokenized_references)['rougeLsum']) # rougeLsum is a ROUGE version specifically tailored for summarization tasks

        elif task_name == "translation":
            results = task_info['results']
            # metric is bleu
            tokenized_references = [" ".join(tokenizer.tokenize(french_ref)) for french_ref, _ in results]
            tokenized_predictions = [" ".join(tokenizer.tokenize(french)) for _, french in results]
            metrics['translation'] = metric.compute(predictions=tokenized_predictions, references=tokenized_references)['bleu']

        else:
            # task is document classification or emotion load detection. The evaluation logic is the same
            results = task_info['results']
            metrics[task_name] = process_clf_tasks_outputs(results, task_name)

    return metrics

In [ ]:
# to visualize the results on 50 samples (approx running time: 8 min):
# {'summarization': 0.13604474561717156, 'translation': 0.06022730874111599, 'classification': 0.3500079051383399, 'causal_language_modeling': 0.8015474639577969, 'emotion_load_detection': 0.4}
metrics = evaluate_benchmark(model, tokenizer)
print(metrics)
display_radar_chart(metrics.values())

## 3. Fine-Tuning Prerequisites

### 3.1 Some Essentials

In [137]:
print(model)

# freeze the model
for p in model.parameters(): p.requires_grad = False

# print layers gradients to ensure all layers are actually frozen
for name, param in model.named_parameters():
    print(name, param.requires_grad)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

### 3.2 Setting Up Adapters

In [138]:
rank = 16
adapters_config = LoraConfig(
      r = rank,                       # rank of lora module
        lora_alpha = rank/2,          # rescales weights parameters: "expressivity" of LoRA parameters
        target_modules=["q_proj", "v_proj"],
        bias="lora_only",
        lora_dropout=0.1,
    )

In [139]:
# plug adapters into the pre-trained model
clm_model = get_peft_model(model, adapters_config)
clm_model

PeftModel(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features=896, o

The following cell is useful to check that the trainable layers are actually those intented: meaning the LoRA layers only.

In [140]:
for name, param in clm_model.named_parameters():
    print(name, param.requires_grad)

base_model.model.model.embed_tokens.weight False
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight False
base_model.model.model.layers.0.self_attn.q_proj.base_layer.bias True
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight True
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight True
base_model.model.model.layers.0.self_attn.k_proj.weight False
base_model.model.model.layers.0.self_attn.k_proj.bias False
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight False
base_model.model.model.layers.0.self_attn.v_proj.base_layer.bias True
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight True
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight True
base_model.model.model.layers.0.self_attn.o_proj.weight False
base_model.model.model.layers.0.mlp.gate_proj.weight False
base_model.model.model.layers.0.mlp.up_proj.weight False
base_model.model.model.layers.0.mlp.down_proj.weight False
ba

## 4. Fine-tuning experiments

In [141]:
num_samples = 100
rank = 4

### 4.1 Sentiment Analysis

Sentiment analysis is a classification task where the goal is to determine the sentiment of a given piece of text. Typically, sentiment is categorized as either `positive` or `negative`, making it a **binary classification task**.  

To perform this task using a decoder-only language model, two main approaches are commonly employed:

- **Generation-Based Approach** consists in prompting with an instruction to generate either "positive" or "negative" based on the sentiment identified in the provided content. The generated output is then compared to the actual class for evaluation.  

- **Classification-Based Approach** involves adding a classification head on top of the LLM's output layer, turning the model into a standard neural classifier. This introduces additional trainable parameters, which can improve classification accuracy. The output is directly suited to binary classification, typically represented as a sigmoid activation output.

### 4.2 Causal Language Modeling

Causal Language Modeling (CLM) is a task where the model learns to predict the next token in a sequence based on the preceding context. This approach is particularly well-suited for decoder-only language models, as it aligns with their architecture, which processes input in a unidirectional manner. Fine-tuning for CLM involves training the model to generate coherent and contextually relevant text by learning from sequential dependencies in the data. In this section, we will explore how to adapt and fine-tune a decoder-only language model for causal language modeling, focusing on practical implementation and key considerations for achieving optimal performance.


The following cell loads and split the corresponding dataset for Causal Language Modeling:

In [142]:
task_info = tasks["causal_language_modeling"]

dataset = load_from_disk(task_info["dataset"])
train_data = dataset["train"].shuffle(seed=42).select(range(1000))
test_data = dataset["test"].shuffle(seed=42).select(range(100))

# data collator: controls batching strategy of data
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [143]:
train_data = train_data.rename_columns({'baseline':'text'})
test_data = test_data.rename_columns({'baseline':'text'})

The following cell shows a data sample:

In [144]:
train_data[4]

{'text': '"Perhaps." Lucian\'s smile never fell from his face. "I noticed your ship jumping in from New Tokyo a few hours ago – I must say, it is enormous! You\'re not thinking of selling, are you?"'}

For now, the dataset contains only textual data. Therefore, we need to tokenize its content to be able to use an LLM. For that, we use the model's tokenizer defined at the beggining of the notebook:

In [145]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

def tokenize_function(data_sample):
    return tokenizer(data_sample["text"], truncation=True, return_tensors="pt", padding="max_length", max_length=128)

tokenized_train_data = train_data.map(tokenize_function, batched=True)
tokenized_train_data = tokenized_train_data.remove_columns(["text"])

tokenized_test_data = test_data.map(tokenize_function, batched=True)
tokenized_test_data = tokenized_test_data.remove_columns(["text"])

In the following cell, we visualize the structure of the tokenized dataset and the previous data sample in its tokenized version:

In [146]:
tokenized_train_data
print(tokenized_train_data[4])

{'input_ids': [1, 31476, 1189, 13784, 1103, 594, 15289, 2581, 11052, 504, 806, 3579, 13, 330, 40, 13686, 697, 8284, 29002, 304, 504, 1532, 26194, 264, 2421, 4115, 4134, 1365, 358, 1969, 1977, 11, 432, 374, 22399, 0, 1446, 2299, 537, 7274, 315, 11236, 11, 525, 498, 7521, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Now, we can observe the difference in the dataset column names: we replaced `text` by `input_ids`, the token indexes in the vocabulary, and `attention_mask`. The data is now ready to be fed to the LLM for fine-tuning.

In [153]:
training_args = TrainingArguments(
    output_dir="clm_finetuning",
    eval_strategy="epoch",
    logging_steps=10,
    learning_rate=5e-5, 
    weight_decay=0.01,
    num_train_epochs=5,
    seed=42,
    eval_accumulation_steps=1,
    prediction_loss_only=True,
    max_grad_norm=1.0,
)

In [154]:
trainer = Trainer(
    model=clm_model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    data_collator=data_collator,
)

Run training (approx running time: 8 min)

In [156]:
trainer.train()

  2%|▏         | 10/625 [00:09<09:13,  1.11it/s]

{'loss': 3.4568, 'grad_norm': 1.414170265197754, 'learning_rate': 4.92e-05, 'epoch': 0.08}


  3%|▎         | 20/625 [00:18<09:28,  1.06it/s]

{'loss': 3.4638, 'grad_norm': 1.6173585653305054, 'learning_rate': 4.8400000000000004e-05, 'epoch': 0.16}


  5%|▍         | 30/625 [00:27<09:11,  1.08it/s]

...

{'loss': 3.2594, 'grad_norm': 1.263951063156128, 'learning_rate': 3.16e-05, 'epoch': 1.84}


 38%|███▊      | 240/625 [03:43<05:53,  1.09it/s]

{'loss': 3.3635, 'grad_norm': 1.7804468870162964, 'learning_rate': 3.08e-05, 'epoch': 1.92}


 39%|███▉      | 244/625 [03:47<05:40,  1.12it/s]

KeyboardInterrupt: 

### 4.3 Text Summarization

Text summarization is the task of generating a concise and coherent summary that captures the essential information from a longer piece of text. It is a challenging task that requires the model to understand the context, identify key points, and rephrase information effectively. For decoder-only language models, summarization can be approached as a sequence-to-sequence generation task where the input is the original text, and the output is its summary. In this section, we will fine-tune a language model for text summarization, covering practical techniques, data preparation, and strategies to improve the quality and relevance of the generated summaries.

## 5. Benchmark Fine-Tuned Models: Conclusions